In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/americannlp-task-2-dataset/maya-test.tsv
/kaggle/input/americannlp-task-2-dataset/bribri-dev.tsv
/kaggle/input/americannlp-task-2-dataset/nahuatl_omitlan-test.tsv
/kaggle/input/americannlp-task-2-dataset/nahuatl_omitlan-dev.tsv
/kaggle/input/americannlp-task-2-dataset/guarani-train.tsv
/kaggle/input/americannlp-task-2-dataset/guarani-dev.tsv
/kaggle/input/americannlp-task-2-dataset/guarani-test.tsv
/kaggle/input/americannlp-task-2-dataset/maya-train.tsv
/kaggle/input/americannlp-task-2-dataset/bribri-test.tsv
/kaggle/input/americannlp-task-2-dataset/maya-dev.tsv
/kaggle/input/americannlp-task-2-dataset/nahuatl_omitlan-train.tsv
/kaggle/input/americannlp-task-2-dataset/bribri-train.tsv


In [2]:
%%capture
%pip install Dataset
%pip install sacrebleu
%pip install transformers
%pip install sentencepiece
%pip install datasets
%pip install huggingface_hub
%pip install bitsandbytes
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [3]:
# Set environment variable to help with memory allocation
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HUGGINGFACE_TOKEN = user_secrets.get_secret("HUGGINGFACE_TOKEN")

!huggingface-cli login --token $HUGGINGFACE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `basic task` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `basic task`


In [5]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer,SFTConfig
from trl import setup_chat_format
from transformers import (
                          AutoTokenizer,
                          AutoModelForCausalLM,
                          TrainingArguments,
                          BitsAndBytesConfig,
                          pipeline,
                          Trainer,
                          DataCollatorWithPadding,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split
from sacrebleu import corpus_bleu, corpus_chrf

In [6]:
from accelerate import PartialState
device_map={"": PartialState().process_index}

In [7]:
# Load the data
train_df = pd.read_table('/kaggle/input/americannlp-task-2-dataset/maya-train.tsv')
dev_df = pd.read_table('/kaggle/input/americannlp-task-2-dataset/maya-dev.tsv')
test_df = pd.read_table('/kaggle/input/americannlp-task-2-dataset/maya-test.tsv')

In [8]:
X_train = train_df
X_eval = dev_df
X_test_sub = test_df

In [9]:
def generate_prompt(data_point):
    """
    Generate a structured training prompt for a given data point.
    """
    return f"""
Source: {data_point["Source"]}
Instruction: {data_point["Change"]}
Target: {data_point["Target"]}
""".strip()

def generate_test_prompt(data_point):
    return f"""
Source: {data_point["Source"]}
Instruction: {data_point["Change"]}
Provide only the Target sentence nothing else.
Target:""".strip()

In [10]:
# Generate prompts for training and evaluation dataa
X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)

In [11]:
# Convert to datasets
train_data = Dataset.from_pandas(X_train[["text"]])
eval_data = Dataset.from_pandas(X_eval[["text"]])

In [12]:
# Create a new DataFrame for test prompts
test_data = pd.DataFrame({
    "Change": X_eval["Change"],
    "Source": X_eval["Source"]
})
# Generate prompts for test data
X_test = pd.DataFrame(test_data.apply(lambda row: generate_test_prompt(row), axis=1), columns=["text"])

In [13]:
# Set the device (GPU if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:
device

device(type='cuda', index=0)

In [15]:
# Load the pre-trained model and tokenizer
base_model_name = "facebook/xglm-1.7B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.92M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

In [16]:
# Move the model to the GPU
model.to(device)

XGLMForCausalLM(
  (model): XGLMModel(
    (embed_tokens): XGLMScaledWordEmbedding(256008, 2048, padding_idx=1)
    (embed_positions): XGLMSinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0-23): 24 x XGLMDecoderLayer(
        (self_attn): XGLMAttention(
          (k_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (out_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
        )
        (activation_fn): GELUActivation()
        (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear4bit(in_features=2048, out_features=8192, bias=True)
        (fc2): Linear4bit(in_features=8192, out_features=2048, bias=True)
        (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layer_norm): LayerNorm(

In [17]:
# # Define a custom predict function
# def predict(test, model, tokenizer):
#     y_pred = []
    
#     for i in tqdm(range(len(test))):
#         prompt = test.iloc[i]["text"]
#         inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)
        
#         # Generate text using the model directly
#         outputs = model.generate(**inputs, max_length=100, num_beams=4, no_repeat_ngram_size=3).to(device)
        
#         generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True).to(device)
#         transformed_sentence = generated_text.split("Target:")[-1].strip()

#         if transformed_sentence:  
#             y_pred.append(transformed_sentence)
#         else:
#             y_pred.append("ERROR")  # Handle empty outputs
    
#     return y_pred

In [18]:
from tqdm import tqdm
from transformers import pipeline

def clean_prediction(text):
    """
    Extracts the expected transformed sentence from the generated output.
    """
    text = text.strip()

    # Extract text after "Target:" marker
    if "Target:" in text:
        text = text.split("Target:")[-1].strip()

    # Take only the first line to remove unwanted repetitions
    text = text.split("\n")[0].strip()

    return text

def predict(test, model, tokenizer):
    """
    Generate predictions for the test dataset without using a dataset format.
    """
    y_pred = []
    
    # Define pipeline outside loop for efficiency
    pipe = pipeline(task="text-generation", 
                    model=model, 
                    tokenizer=tokenizer, 
                    max_new_tokens=20,  # Limit length to avoid extra output
                    temperature=0.1,  # Make output more deterministic
                    )  

    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["text"]  # Use already pre-generated test prompts
        result = pipe(prompt)
        
        generated_text = result[0]['generated_text']
        transformed_sentence = clean_prediction(generated_text)
        y_pred.append(transformed_sentence if transformed_sentence else "ERROR")  # Handle empty output

    return y_pred

In [19]:
# Evaluate the predictions
def evaluate(y_true, y_pred):
    bleu = corpus_bleu(y_pred, [y_true])
    print(f"BLEU score: {bleu.score:.2f}")

    chrf = corpus_chrf(y_pred, [y_true])
    print(f"chrF score: {chrf.score:.2f}")

    # Accuracy calculation
    accuracy = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {accuracy:.2f}")

    for i in range(min(5, len(y_true))):
        print(f"\nMain Prompt: {X_test.iloc[i]['text']}")
        print(f"Expected Sentence: {y_true[i]}")
        print(f"Prediction: {y_pred[i]}")

In [20]:
# Define LoRA configuration
def find_all_linear_names(model):
    cls = torch.nn.Linear
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)

In [21]:
lora_config = LoraConfig(
    r=4,
    lora_alpha=8,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)

In [22]:
# Set up training arguments
training_arguments = SFTConfig(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    save_steps=1000,
    dataset_text_field="text",
    max_seq_length=512,
    packing=False,
    logging_steps=500,
    learning_rate=2e-4,
    weight_decay=0.01,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="none",
    eval_strategy="steps",
    eval_steps=50,  
)

In [23]:
# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=lora_config,
    args=training_arguments,
)

Converting train dataset to ChatML:   0%|          | 0/594 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/594 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/594 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/594 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/149 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/149 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/149 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/149 [00:00<?, ? examples/s]

In [24]:
# Train the model
trainer.train()

Step,Training Loss,Validation Loss
50,No log,3.599527
100,No log,2.518138
150,No log,2.230969
200,No log,2.150416
250,No log,2.039598
300,No log,2.144372
350,No log,2.223487
400,No log,2.046943
450,No log,1.929887
500,1.976100,1.708423


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=1485, training_loss=1.2435801085398253, metrics={'train_runtime': 966.5091, 'train_samples_per_second': 3.073, 'train_steps_per_second': 1.536, 'total_flos': 867287291904000.0, 'train_loss': 1.2435801085398253})

In [25]:
# Evaluate the model after fine-tuning
y_pred_after_fine_tune = predict(X_test, model, tokenizer)

Device set to use cuda:0
  0%|          | 0/149 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 149/149 [02:16<00:00,  1.09it/s]


In [26]:
# Evaluate the model
y_true = X_eval["Target"]

# Evaluate the model before fine-tuning
print("\nOriginal Model Evaluation After Fine Tuning:")
evaluate(y_true.tolist(), y_pred_after_fine_tune)


Original Model Evaluation After Fine Tuning:
BLEU score: 43.45
chrF score: 70.23
Accuracy: 0.15

Main Prompt: Source: Janalnajen tu k'íiwikil koonol
Instruction: PERSON:1_PL
Provide only the Target sentence nothing else.
Target:
Expected Sentence: Janalnajo'on tu k'íiwikil koonol
Prediction: Ma' jalnajen tu k'íiwikil koonol

Main Prompt: Source: Janalnajen tu k'íiwikil koonol
Instruction: PERSON:2_PL
Provide only the Target sentence nothing else.
Target:
Expected Sentence: Janalnaje'ex tu k'íiwikil koonol
Prediction: Ma' tin k'íiwikil koonol

Main Prompt: Source: Janalnajen tu k'íiwikil koonol
Instruction: PERSON:2_SI
Provide only the Target sentence nothing else.
Target:
Expected Sentence: J-Jaanech tu k'íiwikil koonol
Prediction: Ma' janalnaje'ex tu k'íiwikil koonol

Main Prompt: Source: Táan ek bin ich kool
Instruction: TYPE:NEG
Provide only the Target sentence nothing else.
Target:
Expected Sentence: Ma' táan ek bin ich kooli'
Prediction: Táan ek bin ich kool

Main Prompt: Source:

## Dev Submission

In [27]:
dev_pd = pd.DataFrame(y_pred_after_fine_tune, columns=['Values'])

In [28]:
dev_pd.to_csv('syntax_squad_bribri_dev_output.tsv', sep='\t', index=False, header=False)

In [29]:
from IPython.display import FileLink

FileLink("syntax_squad_bribri_dev_output.tsv")

/kaggle/working/syntax_squad_bribri_dev_output.tsv

## Test Submission

In [30]:
# Create a new DataFrame for test prompts
test_data_sub = pd.DataFrame({
    "Change": X_test_sub["Change"],
    "Source": X_test_sub["Source"]
})
# Generate prompts for test data
X_test_sub = pd.DataFrame(test_data_sub.apply(lambda row: generate_test_prompt(row), axis=1), columns=["text"])

In [31]:
# Evaluate the model before fine-tuning
y_pred_test = predict(X_test_sub, model, tokenizer)

Device set to use cuda:0
  0%|          | 0/310 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 310/310 [05:08<00:00,  1.00it/s]


In [32]:
test_pd = pd.DataFrame(y_pred_test, columns=['Values'])

In [33]:
test_pd.to_csv('syntax_squad_bribri_test_output.tsv', sep='\t', index=False, header=False)

In [34]:
from IPython.display import FileLink

FileLink("syntax_squad_bribri_test_output.tsv")

/kaggle/working/syntax_squad_bribri_test_output.tsv